In [ ]:
import os

import pandas as pd
import plotly.graph_objects as go

In [ ]:
epic = pd.read_parquet(os.environ.get("POETRY_EPIC"))
epic = epic[
    [
        "UCSC_RefGene_Name",
        "UCSC_RefGene_Group",
        "Relation_to_UCSC_CpG_Island",
        "Regulatory_Feature_Group",
        "MAPINFO",
    ]
]

epic = epic[~epic.UCSC_RefGene_Name.isna()]
epic

In [ ]:
samplesheet = pd.read_csv("../data/raw/SampleSheet.csv", index_col=0)
mynorm = pd.read_parquet("../data/processed/CorrectedMyNorms/mynorm.parquet").T
mynorm

In [ ]:
allgenes = pd.read_csv("../Files/COVSpecificGenes.csv", index_col=0).Genes
genes = "|".join(allgenes.tolist())
genes

In [ ]:
epic = epic.sort_values("UCSC_RefGene_Name")
epic = epic[epic["UCSC_RefGene_Name"].str.contains(genes)]

In [ ]:
mynorm = mynorm[set(epic.index).intersection(set(mynorm.columns))]
epic = epic.loc[mynorm.columns]

converted = []
for status in samplesheet.Status:
    if "COVID" in status:
        converted.append("COVID-19")
    elif "Other respiratory infections" in status:
        converted.append("Other respiratory infections")
    else:
        converted.append("Healthy control")

mynorm["Status"] = converted

In [ ]:
for gene in allgenes.unique():

    probes = (
        epic[epic.UCSC_RefGene_Name.str.contains(gene)]
        .sort_values("MAPINFO", ascending=True)
        .index
    )

    fig = go.Figure()

    for group in mynorm.Status.unique():
        temp_mynorm = mynorm[mynorm.Status == group]
        temp_mynorm = temp_mynorm[probes].mean()
        fig.add_trace(
            go.Line(
                x=temp_mynorm.index,
                y=temp_mynorm.values,
                mode="lines+markers",
                name=group,
            )
        )

    fig.update_layout(title=gene)
    fig.show()

In [ ]:
epic[epic.UCSC_RefGene_Name.str.contains("DTX3L|MX1")].sort_values("MAPINFO")